In [ ]:
import repast4py
from repast4py.core import Agent, Context, Schedule, RandomMovementBehavior
from repast4py.space import ContinuousSpace
from repast4py.io import CsvExporter
import pandas as pd
import numpy as np

# Define the CityAgent class
class CityAgent(Agent):
    def __init__(self, unique_id, lon, lat):
        super().__init__(unique_id)
        self.lon = lon
        self.lat = lat

    def step(self, model):
        # Define behavior for each step
        self.move()

    def move(self):
        # Example behavior: move randomly (you can define your own logic here)
        self.lon += np.random.uniform(-0.01, 0.01)
        self.lat += np.random.uniform(-0.01, 0.01)

# Initialize the context and space
context = Context()
space = ContinuousSpace(90, 60)  # Define the space dimensions (90x60 for India's approximate size)

# Load city data
city_data = pd.read_csv('city_data.csv')  # Ensure your CSV has 'lon' and 'lat' columns

# Add agents to the context
for index, row in city_data.iterrows():
    agent = CityAgent(index, row['lon'], row['lat'])
    context.add(agent)
    space.move_to(agent, (row['lon'], row['lat']))

# Define the schedule
schedule = Schedule()
schedule.add(RandomMovementBehavior(context, space))

# Run the simulation
def run_model(steps):
    for i in range(steps):
        schedule.step()
        # Export the state of the agents for analysis (optional)
        CsvExporter.export(context, f'output_step_{i}.csv')

run_model(100)  # Run for 100 steps


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob

# Function to plot agent positions
def plot_agents(step):
    filename = f'output_step_{step}.csv'
    data = pd.read_csv(filename)
    plt.scatter(data['lon'], data['lat'])
    plt.title(f'Agent Positions at Step {step}')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.show()

# Plot positions at a specific step (e.g., step 10)
plot_agents(10)


In [ ]:
import matplotlib.pyplot as plt

# Real-time plotting function
def run_model_with_plotting(steps):
    plt.ion()  # Turn on interactive mode
    fig, ax = plt.subplots()
    scatter = ax.scatter([], [])

    for i in range(steps):
        schedule.step()
        
        # Extract agent positions for plotting
        positions = [(agent.lon, agent.lat) for agent in context.agents]
        lons, lats = zip(*positions)
        
        # Update plot
        scatter.set_offsets(list(zip(lons, lats)))
        ax.set_xlim(min(lons)-0.1, max(lons)+0.1)
        ax.set_ylim(min(lats)-0.1, max(lats)+0.1)
        ax.set_title(f'Agent Positions at Step {i}')
        plt.draw()
        plt.pause(0.1)  # Pause to update the plot

    plt.ioff()  # Turn off interactive mode
    plt.show()

run_model_with_plotting(100)  # Run for 100 steps with real-time plotting


In [ ]:
import plotly.graph_objs as go
import plotly.express as px

# Function to plot agent positions using Plotly
def plot_agents_plotly(step):
    filename = f'output_step_{step}.csv'
    data = pd.read_csv(filename)
    
    fig = px.scatter_geo(data, lon='lon', lat='lat', title=f'Agent Positions at Step {step}')
    fig.update_geos(projection_type="orthographic")
    fig.show()

# Plot positions at a specific step (e.g., step 10)
plot_agents_plotly(10)


In [14]:
import geopandas as gpd
import matplotlib.pyplot as plt
from repast4py import context 
from repast4py import core, space, schedule, logging, random
from repast4py.core import Agent
import pandas as pd
import numpy as np

# Define the CityAgent class
class CityAgent(Agent):
    def __init__(self, unique_id, lon, lat):
        super().__init__(unique_id)
        self.lon = lon
        self.lat = lat

    def step(self, model):
        # Define behavior for each step
        self.move()

    def move(self):
        # Example behavior: move randomly (you can define your own logic here)
        self.lon += np.random.uniform(-0.1, 0.1)
        self.lat += np.random.uniform(-0.1, 0.1)

# Initialize the context and space
context()

# Load city data
city_data = pd.read_csv('city_data.csv')  # Ensure your CSV has 'lon' and 'lat' columns

# Add agents to the context
for index, row in city_data.iterrows():
    agent = CityAgent(index, row['lon'], row['lat'])
    context.add(agent)

# Define the schedule
schedule = Schedule()

# Function to plot agents on the map
def plot_agents(step, gdf_india, ax):
    positions = [(agent.lon, agent.lat) for agent in context.agents]
    lons, lats = zip(*positions)
    ax.clear()
    gdf_india.plot(ax=ax, color='white', edgecolor='black')
    ax.scatter(lons, lats, c='red')
    ax.set_title(f'Agent Positions at Step {step}')
    plt.draw()
    plt.pause(0.1)

# Run the model with real-time plotting
def run_model(steps):
    # Load India map shapefile
    gdf_india = gpd.read_file('india_boundary.shp')  # Replace with your actual shapefile path

    plt.ion()  # Turn on interactive mode
    fig, ax = plt.subplots(figsize=(10, 10))

    for step in range(steps):
        schedule.step()
        plot_agents(step, gdf_india, ax)

    plt.ioff()  # Turn off interactive mode
    plt.show()

run_model(100)  # Run for 100 steps


TypeError: 'module' object is not callable